In [1]:
import numpy as np
import glob
from lb3d_checkpoint_io import *
import copy
import os

import sys

In [ ]:
nx, ny, nz = [64, 64, 128]
Q = 19
gr_out = "restart"

# checkpoint to use fluid data
fluid_timestep = 10000
input_fluid_path = "copy_system/np_5"
nprocs = 4

# checkpoint to use MD data
md_timestep = 10000
input_md_path = "copy_system/np_25"

output_path = f"{input_fluid_path}/graft_particles/{input_md_path}"
os.makedirs(output_path, exist_ok=True)

In [3]:
## READING FLUID CHECKPOINTS ##
checkparams_files = sorted(glob.glob(f"{input_fluid_path}/checkparams*{fluid_timestep:08d}*.xdr"))[0]
fluid_checkpoint_files = sorted(glob.glob(f"{input_fluid_path}/checkpoint*{fluid_timestep:08d}*.xdr"))
checktopo_files = sorted(glob.glob(f"{input_fluid_path}/checktopo*{fluid_timestep:08d}*.xdr"))[0]
md_checkpoint_files = sorted(glob.glob(f"{input_fluid_path}/md-checkpoint*{fluid_timestep:08d}*.xdr"))[0]

curr_check_params = read_checkparams_xdr(checkparams_files)
curr_topo = read_checktopo_xdr(checktopo_files, nprocs = nprocs)
curr_fluid_params = read_checkpoint_xdr(fluid_checkpoint_files, nx, ny, nz, curr_topo, Q)
curr_md_params = read_md_checkpoint_xdr(md_checkpoint_files, use_rotation=True, interaction="ladd", n_spec = 2)
## READING FLUID CHECKPOINTS ##

## READING MD CHECKPOINTS ##
md_checkpoint_files = sorted(glob.glob(f"{input_md_path}/md-checkpoint*{md_timestep:08d}*.xdr"))[0]

new_md_params = read_md_checkpoint_xdr(md_checkpoint_files, use_rotation=True, interaction="ladd", n_spec = 2)
## READING MD CHECKPOINTS ##

print("Checkpoints to be copied have been read")

Checkpoints to be copied have been read


# Parameter file

In [4]:
## PARAMETER CHECKPOINT FILE ##
new_check_params = copy.deepcopy(curr_check_params)
print("New parameter file generated")
## PARAMETER CHECKPOINT FILE ##

New parameter file generated


# Fluid files

In [ ]:
# ## FLUID CHECKPOINT FILE ##
new_fluid_params = curr_fluid_params.copy()
print("New fluid checkpoint file generated")
# ## FLUID CHECKPOINT FILE ##

New fluid checkpoint file generated


# Topology

In [6]:
## TOPOLOGY CHECKPOINT FILE ##
new_topo = copy.deepcopy(curr_topo)
new_topo['cdims'] = [1,1,1]
new_topo['all_ccoords'] = [0,0,0]
new_topo['nprocs'] = 1
print("New mpi topology file generated")
## TOPOLOGY CHECKPOINT FILE ##

New mpi topology file generated


# MD Files

In [7]:
new_md_params = copy.deepcopy(new_md_params)

# Output

In [10]:
## OUTPUTTING NEW CHECKPOINTS ##
uid = np.random.randint(0, 2**31, 1)[0] # Generating random number of a signed FP32 integer

checkparams_file_template = "checkparams_{0}_t{1:08d}-{2:010d}.xdr"
fluid_checkpoint_file_template = "checkpoint_{0}_t{1:08d}-{2:010d}_p{3:06d}.xdr"
checktopo_file_template = "checktopo_{0}_t{1:08d}-{2:010d}.xdr"
md_checkpoint_file_template = "md-checkpoint_{0}_t{1:08d}-{2:010d}.xdr"

output_params_path = output_path + "/" + checkparams_file_template.format(gr_out, fluid_timestep, uid)
output_fluid_path = output_path + "/" + fluid_checkpoint_file_template.format(gr_out, fluid_timestep, uid, 0)
output_topo_path = output_path + "/" + checktopo_file_template.format(gr_out, fluid_timestep, uid)
output_md_check_path = output_path + "/" + md_checkpoint_file_template.format(gr_out, fluid_timestep, uid)

write_checkparams_xdr(output_params_path, new_check_params)
write_checkpoint_xdr(output_fluid_path, new_fluid_params, nx, ny, nz)
write_checktopo_xdr(output_topo_path, new_topo)
write_md_checkpoint_xdr(output_md_check_path, new_md_params["particles"], 
                        use_rotation=True, steps_per_lbe_step=1, interaction="ladd",
                        ladd_props=new_md_params['ladd_data'],
                        n_spec=2)

print(f"Checkpoint output successful!. UID:{uid}")
## OUTPUTTING NEW CHECKPOINTS ##

Checkpoint output successful!. UID:1146934485
